In [1]:
from tqdm import tqdm 
import pandas as pd 
import numpy as np 


In [3]:
df = pd.read_csv('data/train_baseline_helmets.csv')

In [5]:
df.isna().sum()

game_play        0
game_key         0
play_id          0
view             0
video            0
frame            0
nfl_player_id    0
player_label     0
left             0
width            0
top              0
height           0
dtype: int64

In [6]:
# df = df.fillna(99)

In [7]:
df.head(3)

,game_play,game_key,play_id,view,video,frame,nfl_player_id,player_label,left,width,top,height
0,58168_003392,58168,3392,Endzone,58168_003392_Endzone.mp4,290,39947,H72,946,25,293,34
1,58168_003392,58168,3392,Endzone,58168_003392_Endzone.mp4,290,37211,H42,151,25,267,33
2,58168_003392,58168,3392,Endzone,58168_003392_Endzone.mp4,290,38590,H70,810,25,293,35


In [9]:
## cache baseline helmet bounding boxes
det_dict = {}
for i, row in tqdm(df.iterrows()):    
    vid = row['game_play'] + '_' + row['view']
    frame = row['frame']
    p_id = row['nfl_player_id']
        # vid => 58168_003392_Endzone
    if vid not in det_dict:
        det_dict[vid] = {}

    if frame not in det_dict[vid]:
        det_dict[vid][frame] = {}

    if p_id not in det_dict[vid][frame]:
        if 'H' in row['player_label']:
            t = 'home'
        else:
            t = 'v'

        det_dict[vid][frame][p_id] = {'box': [row['left'], row['top'], row['width'], row['height']], 'contact': [], 't': t}



3783616it [08:55, 7063.55it/s]


In [13]:
det_dict['58168_003392_Endzone'][290]

{39947: {'box': [946, 293, 25, 34], 'contact': [], 't': 'home'},
 37211: {'box': [151, 267, 25, 33], 'contact': [], 't': 'home'},
 38590: {'box': [810, 293, 25, 35], 'contact': [], 't': 'home'},
 44822: {'box': [681, 254, 26, 33], 'contact': [], 't': 'home'},
 41944: {'box': [680, 303, 23, 33], 'contact': [], 't': 'v'},
 46137: {'box': [588, 351, 25, 25], 'contact': [], 't': 'v'},
 38556: {'box': [1215, 284, 27, 23], 'contact': [], 't': 'v'},
 45695: {'box': [539, 291, 26, 34], 'contact': [], 't': 'home'},
 40011: {'box': [1090, 262, 25, 31], 'contact': [], 't': 'home'},
 46445: {'box': [244, 296, 25, 32], 'contact': [], 't': 'home'},
 52440: {'box': [694, 180, 24, 32], 'contact': [], 't': 'home'},
 43395: {'box': [1034, 388, 29, 25], 'contact': [], 't': 'v'},
 42565: {'box': [627, 321, 22, 17], 'contact': [], 't': 'home'},
 42386: {'box': [693, 333, 24, 20], 'contact': [], 't': 'v'},
 37084: {'box': [394, 317, 22, 17], 'contact': [], 't': 'v'},
 47944: {'box': [933, 317, 22, 18], 'con

In [12]:
np.save('data/det_dict.npy', det_dict)

In [2]:
trk_df = pd.read_csv('data/train_player_tracking.csv')

In [3]:
trk_df.head(3)

,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58580_001136,58580,1136,44830,2021-10-10T21:08:20.900Z,-108,away,CB,22,61.59,42.60,1.11,0.11,320.33,263.93,0.71,-0.64
1,58580_001136,58580,1136,47800,2021-10-10T21:08:20.900Z,-108,away,DE,97,59.48,26.81,0.23,0.01,346.84,247.16,1.29,0.90
2,58580_001136,58580,1136,52444,2021-10-10T21:08:20.900Z,-108,away,FS,29,72.19,31.46,0.61,0.06,11.77,247.69,0.63,-0.33


In [4]:
trk_df = trk_df[trk_df['step']>-60]
print(trk_df.shape)

(896069, 17)


In [11]:
## cache tracking player position
trk_dict = {}
for i, row in tqdm(trk_df.iterrows()):
    vid = row['game_play']
    p_id = row['nfl_player_id']
    step = row['step']
    idx = f'{vid}_{step}'
    if idx not in trk_dict:
        trk_dict[idx] = {}

    trk_dict[idx][p_id] = {'x': row['x_position'], 'y': row['y_position'], 't': row['team']}

896069it [01:47, 8300.13it/s]


In [12]:
trk_dict['58580_001136_-59']

{53479: {'x': 59.5, 'y': 29.45, 't': 'away'},
 45532: {'x': 57.1, 'y': 31.0, 't': 'home'},
 52554: {'x': 57.91, 'y': 26.31, 't': 'home'},
 52663: {'x': 63.2, 'y': 27.15, 't': 'away'},
 43293: {'x': 51.14, 'y': 26.38, 't': 'home'},
 43424: {'x': 57.29, 'y': 26.33, 't': 'home'},
 42355: {'x': 59.6, 'y': 24.4, 't': 'away'},
 44830: {'x': 60.4, 'y': 43.51, 't': 'away'},
 53876: {'x': 59.54, 'y': 22.68, 't': 'away'},
 52425: {'x': 56.59, 'y': 42.83, 't': 'home'},
 37082: {'x': 57.17, 'y': 29.57, 't': 'home'},
 41242: {'x': 57.37, 'y': 24.76, 't': 'home'},
 42347: {'x': 56.15, 'y': 10.61, 't': 'home'},
 47800: {'x': 59.38, 'y': 26.9, 't': 'away'},
 43330: {'x': 59.81, 'y': 32.83, 't': 'away'},
 40031: {'x': 67.04, 'y': 22.3, 't': 'away'},
 52444: {'x': 74.28, 'y': 30.95, 't': 'away'},
 43351: {'x': 65.02, 'y': 10.78, 't': 'away'},
 46135: {'x': 59.46, 'y': 20.84, 't': 'away'},
 46206: {'x': 57.49, 'y': 22.04, 't': 'home'},
 46119: {'x': 57.29, 'y': 28.1, 't': 'home'},
 52938: {'x': 57.34, 'y

In [13]:
np.save('data/trk_pos.npy', trk_dict)

In [16]:
## cache tracking player metadata
trk_df = pd.read_csv('data/train_player_tracking.csv')
trk_df = trk_df[trk_df.step>-60]
# trk_dict = {}
for i, row in tqdm(trk_df.iterrows()):
    vid = row['game_play']
    idx = row['nfl_player_id']
    idx = f'{vid}_{idx}'
    step = row['step']
    if idx not in trk_dict:
        trk_dict[idx] = {}

    trk_dict[idx][step] = {'s': row['speed'], 'dis': row['distance'], 'dir': row['direction'], 'o': row['orientation'], 'a': row['acceleration'], 'sa': row['sa'], 'x': row['x_position'], 'y': row['y_position'], 't': row['team']}



0it [00:00, ?it/s]

58580_001136_53479


In [19]:
trk_dict['58580_001136_53479'][-59]

{'s': 0.15,
 'dis': 0.02,
 'dir': 61.79,
 'o': 255.95,
 'a': 0.62,
 'sa': -0.15,
 'x': 59.5,
 'y': 29.45,
 't': 'away'}

In [20]:
np.save('data/trk_dict.npy',trk_dict)